<a href="https://colab.research.google.com/github/heldigard/Electiva-Analitica-IA/blob/main/Clase3-Proyecto-Integrador.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🏆 Clase 3: Proyecto Integrador - Análisis de E-Commerce

## Universidad UdeColombia - Especialización en Analítica de Datos

---

### 🎯 Objetivos de esta clase

Al finalizar esta sesión, serás capaz de:

1. **Conectar** a APIs externas y extraer datos en tiempo real
2. **Transformar** datos JSON anidados en DataFrames analizables
3. **Aplicar** todos los conceptos aprendidos en un caso real de e-commerce
4. **Comunicar** hallazgos en un formato ejecutivo profesional

---

## 📋 Contexto del Proyecto

### El Escenario:

> **Eres el nuevo analista de datos de "Fake Store"**, una tienda online en crecimiento.
> 
> La gerencia necesita entender mejor el catálogo de productos para tomar decisiones sobre:
> - 💰 **Estrategia de precios**
> - 📦 **Gestión de inventario**
> - 📣 **Marketing y promociones**

### Tu Misión:

Realizar un análisis exploratorio completo del catálogo de productos y presentar tus hallazgos a la gerencia.

---

## 🌐 Parte 1: ¿Qué es una API?

### Definición Simple:

Una **API** (Application Programming Interface) es como un **mesero en un restaurante**:

```
TÚ (Cliente)          API (Mesero)          SERVIDOR (Cocina)
     │                    │                       │
     │─── Pides datos ───►│                       │
     │                    │─── Solicita datos ───►│
     │                    │                       │
     │                    │◄── Prepara datos ─────│
     │◄── Entrega datos ──│                       │
     │                    │                       │
```

### Tipos de Peticiones HTTP:

| Método | Descripción | Analogía |
|--------|-------------|----------|
| **GET** | Obtener datos | "Tráeme el menú" |
| POST | Enviar datos | "Quiero ordenar esto" |
| PUT | Actualizar datos | "Cambia mi pedido" |
| DELETE | Eliminar datos | "Cancela mi pedido" |

**En este curso solo usaremos GET** (obtener datos).

---

## 🛠️ Configuración Inicial

In [ ]:
# Importar librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests  # Para conectar a APIs

# Configuración visual
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = [10, 6]

# Configuración de pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:,.2f}'.format)

print("✅ ¡Librerías cargadas correctamente!")

---

## 🔌 Parte 2: Conectando a la API

### La API que usaremos:

**Fake Store API** - Una API gratuita y pública de productos de e-commerce.

- **URL Base:** `https://fakestoreapi.com`
- **Endpoint de productos:** `/products`
- **Documentación:** [fakestoreapi.com/docs](https://fakestoreapi.com/docs)

### Paso 1: Hacer la petición a la API

In [ ]:
# URL del endpoint de productos
url = "https://fakestoreapi.com/products"

# Hacer la petición GET
respuesta = requests.get(url)

# Verificar el código de estado
print(f"📡 Código de estado: {respuesta.status_code}")

if respuesta.status_code == 200:
    print("✅ ¡Conexión exitosa! Los datos fueron recibidos.")
else:
    print(f"❌ Error en la conexión. Código: {respuesta.status_code}")

### 📖 Códigos de Estado HTTP Comunes:

| Código | Significado | ¿Qué hacer? |
|--------|-------------|-------------|
| **200** | ✅ Éxito | ¡Todo bien! |
| 400 | Error en tu petición | Revisa la URL |
| 401 | No autorizado | Necesitas credenciales |
| 404 | No encontrado | URL incorrecta |
| 500 | Error del servidor | Intenta más tarde |

### Paso 2: Explorar los datos JSON

In [ ]:
# Convertir la respuesta a JSON (diccionario de Python)
datos_json = respuesta.json()

# ¿Qué tipo de estructura tenemos?
print(f"📊 Tipo de datos: {type(datos_json)}")
print(f"📊 Cantidad de elementos: {len(datos_json)}")

# Ver el primer producto como ejemplo
print("\n📦 Ejemplo de un producto:")
print("-" * 50)
import json
print(json.dumps(datos_json[0], indent=2))

### 🤔 Observación Importante:

¿Notaste que la columna `rating` es un **diccionario anidado**?

```json
"rating": {
    "rate": 3.9,
    "count": 120
}
```

Esto es un problema común al trabajar con APIs. ¡Lo resolveremos!

### Paso 3: Convertir a DataFrame

In [ ]:
# Convertir JSON a DataFrame
df = pd.DataFrame(datos_json)

print(f"✅ DataFrame creado: {df.shape[0]} productos × {df.shape[1]} columnas")
print("\n📋 Primeras filas:")
df.head()

In [ ]:
# Ver la estructura
df.info()

---

## 🧹 Parte 3: Limpieza y Transformación de Datos

### El Reto: La columna `rating` está anidada

Necesitamos separar `rating` en dos columnas:
- `rating_rate` → La puntuación (ej: 4.1)
- `rating_count` → Número de votantes (ej: 120)

In [ ]:
# Método 1: Usar pd.json_normalize (recomendado)
# =============================================

# Paso 1: Normalizar la columna 'rating' en un nuevo DataFrame
rating_df = pd.json_normalize(df['rating'])

# Paso 2: Renombrar las columnas para mayor claridad
rating_df = rating_df.rename(columns={'rate': 'rating_rate', 'count': 'rating_count'})

# Paso 3: Unir este nuevo DataFrame con el original
df = pd.concat([df, rating_df], axis=1)

# Paso 4: Eliminar la columna original 'rating' que ya no necesitamos
df = df.drop(columns=['rating'])

print("✅ Columna 'rating' transformada exitosamente")
print("\n📋 Nuevas columnas:")
df.head()

In [ ]:
# Verificar que la transformación fue exitosa
print("📊 Información del DataFrame limpio:")
df.info()

### ✏️ ACTIVIDAD 1: Exploración Inicial (10 min)

Usa lo aprendido en las clases anteriores para responder:

1. ¿Cuántas categorías de productos hay?
2. ¿Cuál es el precio más alto y más bajo?
3. ¿Cuál es la calificación promedio de los productos?

**Tip:** Usa `.describe()`, `.unique()`, `.value_counts()`

In [ ]:
# Tu código aquí
# ==============

# 1. Categorías únicas
print("📦 Categorías de productos:")
print(df['category'].unique())
print(f"\nTotal de categorías: {df['category'].nunique()}")

In [ ]:
# 2. Estadísticas de precios
print("💰 Estadísticas de Precios:")
df['price'].describe()

In [ ]:
# 3. Calificación promedio
print(f"⭐ Calificación promedio: {df['rating_rate'].mean():.2f}")
print(f"⭐ Calificación mediana: {df['rating_rate'].median():.2f}")

---

## 📊 Parte 4: Análisis Exploratorio Completo

Vamos a responder las preguntas de negocio que nos pidió la gerencia.

### Pregunta 1: ¿Cómo se distribuyen los precios?

In [ ]:
# Histograma de precios
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histograma
axes[0].hist(df['price'], bins=15, color='steelblue', edgecolor='white', alpha=0.7)
axes[0].axvline(df['price'].mean(), color='red', linestyle='--', label=f'Promedio: ${df["price"].mean():.2f}')
axes[0].axvline(df['price'].median(), color='green', linestyle='--', label=f'Mediana: ${df["price"].median():.2f}')
axes[0].set_title('📊 Distribución de Precios', fontsize=13, fontweight='bold')
axes[0].set_xlabel('Precio ($)')
axes[0].set_ylabel('Cantidad de Productos')
axes[0].legend()

# Boxplot
axes[1].boxplot(df['price'], vert=True)
axes[1].set_title('📦 Boxplot de Precios', fontsize=13, fontweight='bold')
axes[1].set_ylabel('Precio ($)')

plt.tight_layout()
plt.show()

# Productos más caros y más baratos
print("\n🔝 Top 5 productos MÁS CAROS:")
print(df.nlargest(5, 'price')[['title', 'price', 'category']])

print("\n🔻 Top 5 productos MÁS BARATOS:")
print(df.nsmallest(5, 'price')[['title', 'price', 'category']])

### 🎓 Interpretación - Pregunta 1:

**Observaciones:**
- La mayoría de productos están en el rango de $0-50
- Hay algunos productos premium con precios altos (outliers)
- La mediana es menor que el promedio → distribución sesgada hacia la derecha

**Insight de negocio:**
> Fake Store es principalmente una tienda de productos **accesibles**. Los productos de alto valor son pocos, probablemente artículos especializados (joyería, electrónica premium).

### Pregunta 2: ¿Cuántos productos hay por categoría?

In [ ]:
# Análisis por categoría
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Cantidad por categoría
conteo_cat = df['category'].value_counts()
colores = sns.color_palette('husl', len(conteo_cat))

axes[0].barh(conteo_cat.index, conteo_cat.values, color=colores)
axes[0].set_title('📦 Cantidad de Productos por Categoría', fontsize=13, fontweight='bold')
axes[0].set_xlabel('Cantidad')

# Agregar etiquetas
for i, v in enumerate(conteo_cat.values):
    axes[0].text(v + 0.2, i, str(v), va='center', fontweight='bold')

# Precio promedio por categoría
precio_cat = df.groupby('category')['price'].mean().sort_values(ascending=True)

axes[1].barh(precio_cat.index, precio_cat.values, color=colores)
axes[1].set_title('💰 Precio Promedio por Categoría', fontsize=13, fontweight='bold')
axes[1].set_xlabel('Precio Promedio ($)')

# Agregar etiquetas
for i, v in enumerate(precio_cat.values):
    axes[1].text(v + 1, i, f'${v:.2f}', va='center', fontweight='bold')

plt.tight_layout()
plt.show()

### 🎓 Interpretación - Pregunta 2:

**Observaciones:**
- Las categorías de **ropa** (men's/women's clothing) dominan el catálogo
- **Joyería** tiene menos productos pero el **precio más alto**
- **Electrónica** tiene precios intermedios

**Insight de negocio:**
> La tienda es principalmente de **moda/ropa**, pero la joyería podría ser más rentable por unidad. Considerar expandir el catálogo de joyería o electrónica de alto margen.

### Pregunta 3: ¿Existe relación entre precio y calificación?

In [ ]:
# Scatter plot: Precio vs Calificación
plt.figure(figsize=(12, 6))

# Crear scatter con colores por categoría
categorias = df['category'].unique()
colores = sns.color_palette('husl', len(categorias))

for i, cat in enumerate(categorias):
    datos_cat = df[df['category'] == cat]
    plt.scatter(datos_cat['price'], datos_cat['rating_rate'], 
                label=cat, alpha=0.7, s=100, color=colores[i])

plt.title('📊 Relación entre Precio y Calificación', fontsize=14, fontweight='bold')
plt.xlabel('Precio ($)')
plt.ylabel('Calificación (⭐)')
plt.legend(title='Categoría', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, alpha=0.3)

# Líneas de referencia
plt.axhline(y=df['rating_rate'].mean(), color='red', linestyle='--', alpha=0.5, label='Calificación promedio')

plt.tight_layout()
plt.show()

# Calcular correlación
correlacion = df['price'].corr(df['rating_rate'])
print(f"\n📈 Correlación precio-calificación: {correlacion:.3f}")
print("\n💡 Interpretación:")
if abs(correlacion) < 0.3:
    print("   → Correlación DÉBIL: El precio NO determina la calificación")
elif abs(correlacion) < 0.7:
    print("   → Correlación MODERADA: Hay cierta relación")
else:
    print("   → Correlación FUERTE: El precio influye significativamente")

### 🎓 Interpretación - Pregunta 3:

**Observaciones:**
- No hay una relación clara entre precio y calificación
- Productos baratos pueden tener buenas calificaciones
- Productos caros no garantizan mejor satisfacción

**Insight de negocio:**
> El precio **no es el factor principal** que determina la satisfacción del cliente. Esto sugiere que otros factores (calidad, descripción, tiempo de envío) son más importantes. **¡Buenas noticias para productos económicos!**

### ✏️ ACTIVIDAD 2: Identificar Productos Estrella (15 min)

**El Reto:**

La gerencia quiere identificar los **"productos estrella"** - aquellos que combinan alta popularidad (muchas valoraciones) con alta calidad (buena calificación).

**Tareas:**

1. Crea una nueva columna `score` calculada como: `score = rating_rate × rating_count`
2. Identifica los 5 productos con mayor `score`
3. Crea una visualización que muestre estos productos estrella

**Usa un chatbot con este prompt:**

```
Tengo un DataFrame 'df' con las columnas:
- title: nombre del producto
- price: precio
- rating_rate: calificación (1-5)
- rating_count: número de valoraciones
- category: categoría

TAREA:
1. Crea una columna 'score' = rating_rate * rating_count
2. Muestra los 5 productos con mayor score en una tabla
3. Crea un gráfico de barras horizontal con los top 5
```

In [ ]:
# 📝 TU CÓDIGO AQUÍ
# =================



---

## 📝 Parte 5: Resumen Ejecutivo

### ¿Qué es un Resumen Ejecutivo?

Un resumen ejecutivo es un documento breve que:
- Presenta los **hallazgos más importantes** de un análisis
- Está escrito para **gerentes que no tienen tiempo** de leer todo
- Incluye **recomendaciones accionables**
- Se enfoca en el **impacto de negocio**, no en la técnica

### Estructura Recomendada:

```
1. CONTEXTO (1-2 oraciones)
   → ¿Qué se analizó y por qué?

2. HALLAZGOS CLAVE (3-5 puntos)
   → Los insights más importantes

3. RECOMENDACIONES (2-3 acciones)
   → ¿Qué debería hacer la empresa?
```

### 📊 Ejemplo de Resumen Ejecutivo para Fake Store:

---

## Resumen Ejecutivo: Análisis del Catálogo de Fake Store

**Fecha:** Enero 2026  
**Analista:** [Tu nombre]  
**Fuente de datos:** Fake Store API (20 productos)

### Contexto
Se realizó un análisis exploratorio del catálogo de productos para identificar oportunidades de mejora en estrategia de precios, inventario y marketing.

### Hallazgos Clave

1. **📦 Composición del Catálogo:** El catálogo está dominado por ropa (60%), seguido de electrónica y joyería. Hay oportunidad de diversificación.

2. **💰 Estrategia de Precios:** La mayoría de productos son accesibles (<$50), pero la joyería tiene el mayor precio promedio. El margen de ganancia podría ser mayor en esta categoría.

3. **⭐ Satisfacción del Cliente:** La calificación promedio es alta (3.9/5). No hay correlación entre precio y calificación, lo que indica que los productos económicos pueden competir en calidad.

4. **🌟 Productos Estrella:** Los productos con mejor combinación de popularidad y calificación son [listar los top 3].

### Recomendaciones

1. **Expandir joyería de alto margen:** Dado el precio promedio superior, considerar ampliar esta categoría.

2. **Promocionar productos estrella:** Usar los productos con mejor "score" en campañas de marketing y posiciones destacadas.

3. **Mantener accesibilidad:** La estrategia de precios bajos funciona - no sacrificar este posicionamiento.

---

### ✏️ ACTIVIDAD FINAL: Tu Resumen Ejecutivo (20 min)

Basándote en tu análisis, escribe tu propio resumen ejecutivo.

**Tip:** Puedes usar un chatbot para ayudarte a redactar, pero asegúrate de que los datos sean correctos y las recomendaciones tengan sentido.

**Prompt sugerido:**
```
Ayúdame a escribir un resumen ejecutivo para la gerencia.

Datos del análisis:
- Dataset: 20 productos de tienda online
- Categorías: ropa de hombre, ropa de mujer, electrónica, joyería
- Precio promedio: $X
- Calificación promedio: X/5
- [Agrega tus hallazgos]

Escríbelo en español, máximo 3 párrafos, con:
- 3 hallazgos clave
- 2 recomendaciones accionables
```

### 📝 TU RESUMEN EJECUTIVO:

---

## Resumen Ejecutivo: [Título de tu análisis]

**Fecha:** [Fecha]  
**Analista:** [Tu nombre]

### Contexto
[Tu contexto aquí]

### Hallazgos Clave
1. [Hallazgo 1]
2. [Hallazgo 2]
3. [Hallazgo 3]

### Recomendaciones
1. [Recomendación 1]
2. [Recomendación 2]

---

---

## 📚 Diccionario de Términos

| Término (Inglés) | Español | Descripción |
|------------------|---------|-------------|
| **API** | API | Interfaz para obtener datos de un servidor |
| **Endpoint** | Punto de conexión | URL específica de la API |
| **Request** | Petición | Solicitud de datos al servidor |
| **Response** | Respuesta | Datos devueltos por el servidor |
| **JSON** | JSON | Formato de datos (texto estructurado) |
| **Status Code** | Código de estado | Número que indica si la petición fue exitosa |
| **DataFrame** | Marco de datos | Tabla de datos en pandas |
| **Nested** | Anidado | Datos dentro de otros datos |
| **Normalize** | Normalizar | Aplanar datos anidados |
| **Correlation** | Correlación | Relación entre dos variables |
| **Outlier** | Valor atípico | Dato que se sale del rango normal |

---

## 📚 Resumen del Curso Completo

### Lo que aprendiste en las 3 clases:

| Clase | Tema | Habilidades Clave |
|-------|------|-------------------|
| **1** | Fundamentos | Prompting efectivo, exploración de datos, limpieza básica |
| **2** | Visualización | 5 tipos de gráficos, interpretación de patrones, insights |
| **3** | Proyecto | APIs, datos anidados, análisis completo, comunicación ejecutiva |

### El Ciclo de Análisis con IA:

```
PLANIFICAR → EJEMPLIFICAR → GENERAR → EJECUTAR → INTERPRETAR → ITERAR
```

### Herramientas de IA Recomendadas:

- 🟢 **ChatGPT** - [chat.openai.com](https://chat.openai.com)
- 🟡 **Qwen** (100% gratis) - [chat.qwen.ai](https://chat.qwen.ai)  
- 🔵 **Google AI Studio** - [aistudio.google.com](https://aistudio.google.com)
- 🟣 **Claude** - [claude.ai](https://claude.ai)
- ⚫ **Grok** - [grok.x.ai](https://grok.x.ai)
- 🔴 **Kimi K2** - [kimi.moonshot.cn](https://kimi.moonshot.cn)

### ⚠️ Recuerda siempre:

> **ANONIMIZA los datos sensibles antes de pegarlos en chatbots gratuitos.**  
> Las versiones gratuitas pueden usar tu información para entrenar sus modelos.

---

## 🎓 ¡Felicitaciones!

Has completado el curso de **Analítica de Datos con IA**.

Ahora tienes las herramientas para:
- ✅ Usar IA como copiloto para análisis de datos
- ✅ Crear visualizaciones que cuenten historias
- ✅ Conectarte a APIs y procesar datos del mundo real
- ✅ Comunicar hallazgos de forma profesional

**Siguiente paso:** Completa el Taller Final para aplicar todo lo aprendido.